In [1]:
import gymnasium as gym
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecMonitor
from stable_baselines3.common.monitor import Monitor
import imageio
import torch
import math
from datetime import datetime
from tqdm import tqdm


In [2]:
# 创建 Ant 环境l
def make_env(env_id, rank=0, seed=0):
    """
    env_id: Gymnasium 环境名
    rank: 每个环境编号（用于seed区分）
    seed: 基础随机种子
    """
    def _init():
        env = gym.make(env_id)
        env.reset(seed=seed + rank)
        env = Monitor(env)
        return env
    return _init

ENV_ID = "Ant-v4"
NUM_ENVS = 1
SEED = 0

## 创建并行环境
env = SubprocVecEnv([make_env(ENV_ID, i, SEED) for i in range(NUM_ENVS)])
env = VecMonitor(env)  # 配合VecMonitor使用

## 创建单环境
# env = gym.make("Ant-v4")
# obs, info = env.reset(seed=34)

suffix_time = datetime.now().strftime("%Y%m%d_%H%M%S")

/home/ruofei/miniforge3/envs/torchrl/lib/python3.9/site-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
/home/ruofei/miniforge3/envs/torchrl/lib/python3.9/site-packages/stable_baselines3/common/vec_env/vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


In [ ]:
# 这里定义网络的隐藏层结构
policy_kwargs = dict(  
    activation_fn=torch.nn.ReLU,   # 这里替换激活函数,只影响隐藏层，不影响输出层
    net_arch=[256, 64],  # actor和critic网络结构相同，但分成两个网络
)


def warm_sin_lr(progress_remaining: float) -> float:
    """
    progress_remaining: 1 -> 0
    假设总共训练T步：
      - 前10% steps: 线性从 1e-5 升到 3e-4 (warm-up)
      - 之后: 按正弦方式从 3e-4 降到 1e-5
    """
    lr_min = 5e-4   # 1e-3, 1e-4取得了不错的效果
    lr_max = 8e-4
    warm_ratio = 0.1  # 10% warm-up

    # progress_remaining=1 -> step=0; progress_remaining=0 -> step=end
    progress_done = 1.0 - progress_remaining

    if progress_done < warm_ratio:
        # warm-up: 线性上升
        return lr_min + (lr_max - lr_min) * (progress_done / warm_ratio)
    else:
        # sin下降：这里重新归一化到[0,1]
        x = (progress_done - warm_ratio) / (1 - warm_ratio)
        return lr_min + (lr_max - lr_min) * math.sin((1 - x) * math.pi / 2)

# 使用 Stable-Baselines3 的 PPO 算法
model = PPO(
    "MlpPolicy",  # 多层感知机作为策略网络
    env,
    verbose=1,
    learning_rate=warm_sin_lr,   # 关键：传入函数
    # learning_rate=0.002,
    n_steps=1024,   # 每次rollout采集2048步数据
    batch_size=64,
    n_epochs=10,     # 对每个采集到的样本进行10次优化。 PPO中多次利用样本，提高数据利用率。注意，因为同一批数据取自同一个策略，所以始终限制策略更新在一个一致的old policy附近
    gamma=0.99,
    gae_lambda=0.95,   # GAE参数，控制偏差与方差的权衡
    tensorboard_log="./ant_tb/",   # TensorBoard 日志目录
    policy_kwargs=policy_kwargs,  # 关键
)

# 训练模型, total_timesteps自行调整
model.learn(total_timesteps=800000, tb_log_name="ppo_256_64")
# 保存模型
model.save(suffix_time+"_ppo_ant")
env.close()

Using cuda device
Logging to ./ant_tb/ppo_256_64_2


/home/ruofei/miniforge3/envs/torchrl/lib/python3.9/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.8     |
|    ep_rew_mean     | -39.3    |
| time/              |          |
|    fps             | 700      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 44.6        |
|    ep_rew_mean          | -45.1       |
| time/                   |             |
|    fps                  | 588         |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.024397857 |
|    clip_fraction        | 0.197       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.3       |
|    explained_variance   | -0.00347    |
|    learning_rate        | 0.

In [ ]:
policy_kwargs = dict(  
    activation_fn=torch.nn.ReLU,   # 这里替换激活函数,只影响隐藏层，不影响输出层
    net_arch=[256, 64],  # actor和critic网络结构相同，但分成两个网络
)

model2 = SAC(  # 默认[256, 256], actor和critic网络结构相同，但分成两个网络
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    buffer_size=1_000_000,      # 经验回放缓冲区大小. 这个参数PPO没有
    batch_size=256,             # 默认256
    tau=0.005,                  # 软更新系数
    gamma=0.99,                 # 折扣因子
    train_freq=1,               # 每步都训练，采集多少个环境步的数据后训练一次
    gradient_steps=1,           # 对replayBuffer中读取到的batch，进行多少次梯度下降更新
    tensorboard_log="./ant_tb/",   # 日志目录
)

# 训练模型, total_timesteps自行调整
model2.learn(total_timesteps=800000, tb_log_name="sac_256_64")
# 保存模型
model2.save(suffix_time + "_sac_ant")
env.close()

In [ ]:
!tensorboard --logdir ./ppo_ant_tb/

# then 然后浏览器打开 http://localhost:6006

In [ ]:

# 使用可视化界面记录显示PP0测试结果
# 加载模型
model = PPO.load(suffix_time+"_ppo_ant")
# 创建测试环境
env = gym.make("Ant-v4", render_mode="human")

for i in range(5):
    # 测试模型
    state, info = env.reset()
    cum_reward = 0
    for _ in tqdm(range(1500)):
        env.render()
        action, _ = model.predict(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        cum_reward += reward
        if terminated or truncated:
            print("累积奖励: ", cum_reward)
            break
            
        state = next_state

env.close()

In [ ]:

# 使用可视化界面记录显示SAC测试结果
# 加载模型
model = SAC.load(suffix_time+"_sac_ant")
# 创建测试环境
env = gym.make("Ant-v4", render_mode="human")

for i in range(5):
    # 测试模型
    state, info = env.reset()
    cum_reward = 0
    for _ in tqdm(range(1500)):
        env.render()
        action, _ = model.predict(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        cum_reward += reward
        if terminated or truncated:
            print("累积奖励: ", cum_reward)
            break
            
        state = next_state

env.close()

In [ ]:
# 保存为视频

model = PPO.load("ppo_ant")
# 创建测试环境
env = gym.make("Ant-v4", render_mode="rgb_array")
# 存储每帧图像
frames = []
# 测试模型
obs, info = env.reset()
for _ in range(1000):
    env.render()
    frames.append(env.render())  # 捕获帧
    action, _ = model.predict(obs)
    next_state, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()
    else:
        obs = next_state

env.close()

# 保存为视频
imageio.mimsave("./ppo_ant_video.mp4", frames, fps=30)